In [1]:
import os
import csv
import json
import pandas as pd
from collections import Counter, defaultdict

It's doing better than before on shapes.. and it can now build diamonds sometimes.. When looking at the visualizer, I saw it messing up square as rectangle quite a bit. Also, for diamonds the size was always 3 for the examples I saw. Can you check with the visualizer that when we detect a square it's actually a square and for diamonds it's making of varied sizes? Also, we need the orientation breakdown.

In [2]:
home=%pwd
#Nebula original on clarif q
# csv_path = home + '/nebula_org_clarif_output_foranalysis.csv'
#Nebula finetuned on clarif q
# csv_path = home + '/nebula_ft_clarif_output_foranalysis.csv'
#Nebulipa with structure on clarif q
# csv_path = home + '/nebulipa_clarif_withstructure_foranalysis.csv'
#Nebulipa withOUT structure on clarif q
csv_path = home + '/nebulipa_clarif_withoutstructure_foranalysis.csv'

In [ ]:
# with open(csv_path, newline='') as f:
#     reader = csv.reader(f)
#     data = list(reader)

In [3]:
df = pd.read_csv(csv_path)

In [4]:
df.shape

(861, 8)

In [5]:
df.head(10)

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,In_bounds,Full_struct_corr
0,tower,1,1,1,0.0,NaN,1,0
1,tower,1,1,1,0.0,NaN,1,0
2,tower,1,1,1,1.0,NaN,1,1
3,tower,1,1,1,1.0,NaN,1,1
4,tower,1,1,1,1.0,NaN,1,1
5,tower,1,1,1,1.0,NaN,1,1
6,tower,1,1,1,1.0,NaN,1,1
7,diamond,1,0,0,NaN,NaN,1,0
8,diamond,1,0,0,NaN,NaN,1,0
9,diamond,1,0,0,NaN,NaN,1,0


In [6]:
Counter(df['Shape'].tolist())

Counter({'tower': 168,
         'diagonal': 168,
         'row': 168,
         'square': 152,
         'rectangle': 129,
         'diamond': 52,
         'cube': 24})

In [11]:
Counter(df['Shape_corr'].tolist())

Counter({1: 590, 0: 271})

In [8]:
Counter(df['Size_corr'].tolist())

Counter({1: 514, 0: 347})

In [18]:
Counter(df['Color'].tolist())

Counter({1: 857, 0: 4})

In [ ]:
Counter(df['Full_struct_corr'].tolist())

In [ ]:
#Counter(df['Loc_corr'].tolist())

In [19]:
#which samples didn't get color right? Only when the moves were botched
df.loc[df['Color'] == 0]

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,In_bounds,Full_struct_corr
26,square,0,1,1,0.0,NaN,1,0
53,square,0,1,1,0.0,NaN,1,0
54,square,0,0,0,0.0,NaN,1,0
55,square,0,0,0,0.0,NaN,1,0


In [9]:
#what the breakdown of correct and incorrect shapes?
#Correct:
Counter(df.loc[df['Shape_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 168,
         'row': 166,
         'rectangle': 83,
         'square': 77,
         'diagonal': 76,
         'cube': 20})

In [10]:
#what the breakdown of correct and incorrect shapes?
#INorrect:
Counter(df.loc[df['Shape_corr'] == 0, 'Shape'].tolist())

Counter({'diagonal': 92,
         'square': 75,
         'diamond': 52,
         'rectangle': 46,
         'cube': 4,
         'row': 2})

In [ ]:
#Incorrect:
Counter(df.loc[df['Shape_corr'].isin([0,1]), 'Shape'].tolist())

In [12]:
#what the breakdown of correct and incorrect sizes?
#Correct:
Counter(df.loc[df['Size_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 168,
         'row': 164,
         'square': 77,
         'diagonal': 73,
         'rectangle': 18,
         'cube': 14})

In [ ]:
#what the breakdown of correct and incorrect locations?
#Correct:
Counter(df.loc[df['Loc_corr'] == 1, 'Shape'].tolist())

In [ ]:
Counter(df[(df['Size_corr'] == 0)&(df['Shape_corr'] == 1)]['Shape'].tolist())

In [13]:
Counter(df[(df['Loc_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 142,
         'row': 135,
         'rectangle': 77,
         'diagonal': 66,
         'square': 58,
         'cube': 15})

In [14]:
Counter(df[(df['Loc_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'row': 70,
         'tower': 64,
         'rectangle': 41,
         'diagonal': 14,
         'square': 7,
         'cube': 4})

In [ ]:
#Shape corr and location specified
Counter(df[(df['Shape_corr'] == 1)&(df['Loc_corr'].isin([1,0]))]['Shape'].tolist())

In [15]:
#Shape corr and is in bounds
Counter(df[(df['Shape_corr'] == 1)&(df['In_bounds'] == 0)&(df['Loc_corr'].isin([0, 1]))]['Shape'].tolist())

Counter({'rectangle': 2, 'row': 2, 'diagonal': 2})

In [ ]:
#Shape corr and is in bounds
Counter(df[(df['Shape_corr'] == 1)&(df['In_bounds'] == 0)&(df['Loc_corr'].isin([1]))]['Shape'].tolist())

In [16]:
Counter(df[(df['Orient_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'rectangle': 79, 'square': 72})

In [17]:
Counter(df[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'rectangle': 75, 'square': 68})

In [ ]:
#get orientation types
indices = df.index[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)].tolist()

In [ ]:
json_path = home +  '/llama_aug_synth_function_output_v2.json'

with open(json_path, 'r') as j:
    jfile = json.load(j)
    samples = jfile

In [ ]:
orient_dict = defaultdict(list)

for sample in samples:
    shape = sample['shape'] 
    if sample['index'] in indices:
        orient_dict[shape].append(sample['orient'])
        

In [ ]:
Counter(orient_dict['square'])

In [ ]:
Counter(orient_dict['diamond'])

#### for each incorrect square, get the instruction size, then check if square

In [ ]:
import functions as fn

In [ ]:
rec_squares = [['index', 'inst_size', 'rec_size']]

In [ ]:
#get squares that are wrong because they are rectangles -- maybe try "is rectangle" in these cases
indices = df.index[df['Shape'].isin(['square'])& df['Shape_corr'].isin([0])].tolist()

In [ ]:
len(indices)

In [ ]:
indices[:3]

In [ ]:
# json_path = home + '/llama_synth_function_output.json'

# with open(json_path, 'r') as j:
#     jfile = json.load(j)
#     samples = jfile

In [ ]:
for ind in indices:
    shape = samples[ind]['shape']
    inst = samples[ind]['size']
    seq = samples[ind]['net_seq']
    rec_size = None
    is_rectu = fn.is_rectangle_unfilled(seq)
    is_rect = fn.is_rectangle(seq)
    if is_rect:
        rec_size = is_rect[1]
    elif is_rectu:
        rect_size = is_rectu[1]
    rec_squares.append([ind, inst, rec_size])



In [ ]:
rec_squares

In [ ]:
len(rec_squares)

In [ ]:
import csv
square_path = home + '/rec_squares.csv'

In [ ]:
with open(square_path, 'w', newline='') as cfile:
    writer = csv.writer(cfile)
    writer.writerows(rec_squares)

In [ ]:
indices = df.index[(df['Shape'].isin(['square']))&(df['Shape_corr'] == 1)].tolist()

In [ ]:
indices